In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from mapie.regression import MapieRegressor
import matplotlib.pyplot as plt
from mapie.metrics import regression_coverage_score
from mapie.metrics import regression_mean_width_score
from sklearn.ensemble import GradientBoostingRegressor
from mapie.regression import MapieQuantileRegressor
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('GenericWellDataPrepped.csv')

In [3]:
dff = pd.read_csv('GenericWellDataPrepped.csv').copy()

In [4]:
df.head(10)

,Well Index,TrueVerticalDepth_FT,MeasuredDepth_FT,UpperPerforation_FT,LowerPerforation_FT,PerforationInterval_FT,LateralLength_FT,ProppantLoad_LBSPerGAL,ProppantIntensity_LBSPerFT,TotalProppant_LBS,...,CumProd_BOE,CumProd_BOEPer1000FT,CumGas_MCF,CumGas_MCFPer1000FT,CumProd_MCFE,CumProd_MCFEPer1000FT,CumOil_BBL,CumOil_BBLPer1000FT,CumWater_BBL,First3MonthProd_BOEPer1000FT
0,1,11890.0,15995.0,11878.0,15811.0,3933.0,3718.0,4.445714,1802.428571,1.429378e+07,...,153536,39038.0,856573,217791.0,921216,234227.0,10774,2739.0,2182,244.0
1,2,9141.0,13493.0,9470.0,13400.0,3930.0,4076.0,0.770000,672.000000,2.641750e+06,...,170139,43292.0,491924,125171.0,1020832,259754.0,88151,22430.0,141001,4894.0
2,3,9688.0,13824.0,10082.0,13739.0,3657.0,3943.0,0.990000,1094.000000,4.000020e+06,...,99049,27085.0,102078,27913.0,594295,162509.0,82036,22433.0,175835,868.0
3,4,9187.0,13542.0,9451.0,13436.0,3985.0,4145.0,0.470000,560.000000,2.233000e+06,...,195873,49153.0,337319,84647.0,1175237,294915.0,139653,35045.0,40064,6591.0
4,5,7935.0,12859.0,8478.0,12690.0,4212.0,4564.0,0.841429,1195.000000,5.033010e+06,...,137857,32730.0,200840,47683.0,827145,196378.0,104384,24783.0,310369,2852.0
5,6,9048.0,13056.0,9340.0,12935.0,3595.0,3815.0,0.430000,452.000000,1.626000e+06,...,120312,33466.0,237741,66131.0,721870,200798.0,80688,22445.0,64662,3775.0
6,7,10472.0,14522.0,11038.0,14467.0,3429.0,3779.0,0.840000,1215.000000,4.166280e+06,...,116709,34036.0,194882,56833.0,700257,204216.0,84229,24564.0,141109,2004.0
7,8,9285.0,13464.0,9437.0,13314.0,3877.0,4004.0,0.470000,392.000000,1.518162e+06,...,90656,23383.0,248095,63991.0,543937,140298.0,49307,12718.0,46884,2306.0
8,9,9315.0,13624.0,9707.0,13448.0,3741.0,4108.0,0.560000,583.000000,2.182000e+06,...,93179,24907.0,146692,39212.0,559072,149444.0,68730,18372.0,108360,4477.0
9,10,8266.0,13922.0,8505.0,13835.0,5330.0,5460.0,0.930000,1001.000000,5.333000e+06,...,489556,91849.0,646210,121240.0,2937338,551095.0,381855,71643.0,942247,4295.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 89 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Well Index                               16707 non-null  int64  
 1   TrueVerticalDepth_FT                     16707 non-null  float64
 2   MeasuredDepth_FT                         16707 non-null  float64
 3   UpperPerforation_FT                      16707 non-null  float64
 4   LowerPerforation_FT                      16707 non-null  float64
 5   PerforationInterval_FT                   16707 non-null  float64
 6   LateralLength_FT                         16707 non-null  float64
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64
 9   TotalProppant_LBS                        16707 non-null  float64
 10  TotalWaterPumped_GAL                     16707

In [6]:
#df = df.iloc[:, :44]
#df.drop(df.columns[26], axis=1, inplace=True)
#print(df.info())

In [7]:
df_cleaned = df.copy()

In [8]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 89 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Well Index                               16707 non-null  int64  
 1   TrueVerticalDepth_FT                     16707 non-null  float64
 2   MeasuredDepth_FT                         16707 non-null  float64
 3   UpperPerforation_FT                      16707 non-null  float64
 4   LowerPerforation_FT                      16707 non-null  float64
 5   PerforationInterval_FT                   16707 non-null  float64
 6   LateralLength_FT                         16707 non-null  float64
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64
 9   TotalProppant_LBS                        16707 non-null  float64
 10  TotalWaterPumped_GAL                     16707

In [9]:
# Splitting data into Water, Gas, and Oil 
# Splitting data into 3 month and 6 month
y_w_3 = df_cleaned['First3MonthWater_BBL']
y_g_3 = df_cleaned['First3MonthGas_MCF']
y_o_3 = df_cleaned['First3MonthOil_BBL']
y_w_6 = df_cleaned['First6MonthWater_BBL']
y_g_6 = df_cleaned['First6MonthGas_MCF']
y_o_6 = df_cleaned['First6MonthOil_BBL']

In [10]:
X=df_cleaned
X = X.drop('InitialProductionDate',axis=1)
X = X.drop('InitialProductionMonth',axis=1)
X = X.drop('DrillingStartDate',axis=1)
X = X.drop('DrillingCompletionDate',axis=1)
for col in X.columns:
    if pd.api.types.is_object_dtype(X[col]):
        X[col] = X[col].str.replace(',', '')
        X[col] = X[col].str.replace(' ', '')
        X[col] = X[col].astype(float)
X.head()

,Well Index,TrueVerticalDepth_FT,MeasuredDepth_FT,UpperPerforation_FT,LowerPerforation_FT,PerforationInterval_FT,LateralLength_FT,ProppantLoad_LBSPerGAL,ProppantIntensity_LBSPerFT,TotalProppant_LBS,...,CumProd_BOE,CumProd_BOEPer1000FT,CumGas_MCF,CumGas_MCFPer1000FT,CumProd_MCFE,CumProd_MCFEPer1000FT,CumOil_BBL,CumOil_BBLPer1000FT,CumWater_BBL,First3MonthProd_BOEPer1000FT
0,1,11890.0,15995.0,11878.0,15811.0,3933.0,3718.0,4.445714,1802.428571,1.429378e+07,...,153536,39038.0,856573,217791.0,921216,234227.0,10774,2739.0,2182,244.0
1,2,9141.0,13493.0,9470.0,13400.0,3930.0,4076.0,0.770000,672.000000,2.641750e+06,...,170139,43292.0,491924,125171.0,1020832,259754.0,88151,22430.0,141001,4894.0
2,3,9688.0,13824.0,10082.0,13739.0,3657.0,3943.0,0.990000,1094.000000,4.000020e+06,...,99049,27085.0,102078,27913.0,594295,162509.0,82036,22433.0,175835,868.0
3,4,9187.0,13542.0,9451.0,13436.0,3985.0,4145.0,0.470000,560.000000,2.233000e+06,...,195873,49153.0,337319,84647.0,1175237,294915.0,139653,35045.0,40064,6591.0
4,5,7935.0,12859.0,8478.0,12690.0,4212.0,4564.0,0.841429,1195.000000,5.033010e+06,...,137857,32730.0,200840,47683.0,827145,196378.0,104384,24783.0,310369,2852.0


In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16707 entries, 0 to 16706
Data columns (total 85 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Well Index                               16707 non-null  int64  
 1   TrueVerticalDepth_FT                     16707 non-null  float64
 2   MeasuredDepth_FT                         16707 non-null  float64
 3   UpperPerforation_FT                      16707 non-null  float64
 4   LowerPerforation_FT                      16707 non-null  float64
 5   PerforationInterval_FT                   16707 non-null  float64
 6   LateralLength_FT                         16707 non-null  float64
 7   ProppantLoad_LBSPerGAL                   16707 non-null  float64
 8   ProppantIntensity_LBSPerFT               16707 non-null  float64
 9   TotalProppant_LBS                        16707 non-null  float64
 10  TotalWaterPumped_GAL                     16707

# Train and Test sets need to be processed differently for conformal predictions
## This breaks the data into more than just training and testing, allowing for model calibration data
# Processing for water data
## The model seems to struggle with accurately predicting these values, and the conformal prediction intervals come close, but might need an adjusted alpha value. I believe we knew water would be a struggle to predict though

In [12]:
X_train, X_rest1, y_train_w_3, y_rest_w_3_1 = train_test_split(X,y_w_3, test_size=2000, random_state=99)
X_test, X_rest2, y_test_w_3, y_rest_w_3_2 = train_test_split(X_rest1, y_rest_w_3_1, test_size = 1500, random_state=99)
X_calib, X_new, y_calib_w_3, y_new_w_3 = train_test_split(X_rest2, y_rest_w_3_2, test_size = 500, random_state=99)

In [13]:
print("Data sizes: train: %i, test: %i, calibration: %i, new: %i" % (
len(X_train), len(X_test), len(X_calib), len(X_new)
))

Data sizes: train: 14707, test: 500, calibration: 1000, new: 500


In [14]:
param_grid = {
    'learning_rate': [0.01, 0.75, 0.1, 0.25],
    'n_estimators': [300, 400, 500, 750],
    'max_depth': [5, 7, 9, 11],
    'alpha': [0.1, 0.5, 0.75, 0.999]
}
gb_mod_t = XGBRegressor(random_state=99)
grid_search = GridSearchCV(estimator=gb_mod_t, param_grid=param_grid, cv = 2, scoring='r2')

In [15]:
gb_mod_3 = XGBRegressor(learning_rate=0.1, n_estimators= 300, max_depth = 9, random_state=99, alpha = 0.99)
gb_mod_3.fit(X_train, y_train_w_3)
print("XG Boost (default parameters) Train R2: ", gb_mod_3.score(X_train, y_train_w_3))
print("XG Boost (default parameters) Test R2: ", gb_mod_3.score(X_test, y_test_w_3))

XG Boost (default parameters) Train R2:  0.9999996809059956
XG Boost (default parameters) Test R2:  0.9995616744414089


In [16]:
y_pred = gb_mod_3.predict(X_test)
mae=mean_absolute_error(y_test_w_3,y_pred)
print(round(mae,2))

486.86


In [17]:
mapie_reg = MapieRegressor(estimator=gb_mod_3,cv="prefit")
mapie_reg = mapie_reg.fit(X_calib, y_calib_w_3)

In [18]:
y_pred,y_pis = mapie_reg.predict(X_new,alpha=1/3)

In [28]:
# Loop through the first 5 wells
for i in range(5):
    well_index = X_new.iloc[i]['Well Index']
    print(f"Well {well_index}")
    #print(X_new_o_3.iloc[i])  # Print the features of the i-th well
    print(f"Predicted water: {y_pred[i]:.2f}")  # Print the predicted oil
    
    interval = y_pis[i].flatten()  # Flatten the interval if it's not already a 1D array
    print(f"67% interval: [{interval[0]:.2f};{interval[1]:.2f}]")  # Print the 67% prediction interval
    print("---") # Separate the Well data

Well 2428.0
Predicted water: 861.87
67% interval: [543.37;1180.37]
---
Well 3988.0
Predicted water: 77928.90
67% interval: [77610.40;78247.40]
---
Well 16202.0
Predicted water: 137965.59
67% interval: [137647.09;138284.09]
---
Well 5971.0
Predicted water: 60672.37
67% interval: [60353.87;60990.87]
---
Well 15053.0
Predicted water: 18.08
67% interval: [-300.42;336.58]
---


# Processing for oil data
## These results from the model look quite good, and the intervals provided within the conformal predicting capture all of the actual observed values for the first five wells

In [20]:
X_train_o_3, X_rest_o_3_1, y_train_o_3, y_rest_o_3_1 = train_test_split(X,y_o_3, test_size=2000, random_state=99)
X_test_o_3, X_rest_o_3_2, y_test_o_3, y_rest_o_3_2 = train_test_split(X_rest_o_3_1, y_rest_o_3_1, test_size = 1500, random_state=99)
X_calib_o_3, X_new_o_3, y_calib_o_3, y_new_o_3 = train_test_split(X_rest_o_3_2, y_rest_o_3_2, test_size = 500, random_state=99)

In [21]:
print("Data sizes: train: %i, test: %i, calibration: %i, new: %i" % (
len(X_train_o_3), len(X_test_o_3), len(X_calib_o_3), len(X_new_o_3)
))

Data sizes: train: 14707, test: 500, calibration: 1000, new: 500


In [22]:
gb_mod_7 = XGBRegressor(learning_rate=0.075, n_estimators= 400, max_depth = 7, random_state=99, alpha = 0.5)
gb_mod_7 = gb_mod_7.fit(X_train_o_3, y_train_o_3)
print("XG Boost (default parameters) Train R2: ", gb_mod_7.score(X_train_o_3, y_train_o_3))
print("XG Boost (default parameters) Test R2: ", gb_mod_7.score(X_test_o_3, y_test_o_3))

XG Boost (default parameters) Train R2:  0.9999939058504842
XG Boost (default parameters) Test R2:  0.8839158200493888


In [23]:
y_pred_o_3 = gb_mod_7.predict(X_test_o_3)
mae_o_3=mean_absolute_error(y_test_o_3,y_pred_o_3)
print(round(mae_o_3,2))

1139.78


In [24]:
mapie_reg_o_3 = MapieRegressor(estimator=gb_mod_7,cv="prefit")
mapie_reg_o_3 = mapie_reg_o_3.fit(X_calib_o_3, y_calib_o_3)

In [25]:
y_pred_o_3,y_pis_o_3 = mapie_reg_o_3.predict(X_new_o_3,alpha=1/3)

In [26]:
# Loop through the first 5 wells
for i in range(5):
    well_index = X_new_o_3.iloc[i]['Well Index']
    print(f"Well {well_index}")
    #print(X_new_o_3.iloc[i])  # Print the features of the i-th well
    print(f"Predicted oil: {y_pred_o_3[i]:.2f}")  # Print the predicted oil
    
    interval_o_3 = y_pis_o_3[i].flatten()  # Flatten the interval if it's not already a 1D array
    print(f"67% interval: [{interval_o_3[0]:.2f};{interval_o_3[1]:.2f}]")  # Print the 67% prediction interval
    print("---") # Separate the Well data

Well 2428.0
Predicted oil: 962.14
67% interval: [833.95;1090.32]
---
Well 3988.0
Predicted oil: 25870.26
67% interval: [25742.07;25998.45]
---
Well 16202.0
Predicted oil: 60724.15
67% interval: [60595.96;60852.33]
---
Well 5971.0
Predicted oil: 45463.68
67% interval: [45335.49;45591.86]
---
Well 15053.0
Predicted oil: 49698.39
67% interval: [49570.21;49826.58]
---
